In [1]:
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
from matplotlib import style

In [2]:
# -*- coding: utf-8 -*-
code_df = pd.read_html('http://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=13', header=0)[0] 
# 종목코드가 6자리이기 때문에 6자리를 맞춰주기 위해 설정해줌 
code_df.종목코드 = code_df.종목코드.map('{:06d}'.format) 
# 우리가 필요한 것은 회사명과 종목코드이기 때문에 필요없는 column들은 제외해준다. 
code_df = code_df[['회사명', '종목코드']] 
# 한글로된 컬럼명을 영어로 바꿔준다. 
code_df = code_df.rename(columns={'회사명': 'name', '종목코드': 'code'})
code_df.head()


,name,code
0,DSR,155660
1,GS,078930
2,GS글로벌,001250
3,HDC현대산업개발,294870
4,LG이노텍,011070


In [3]:
def get_url(item_name, code_df): 
    code = code_df.query("name=='{}'".format(item_name))['code'].to_string(index=False) 
    code =  code.strip()
    url = "http://finance.naver.com/item/sise_day.nhn?code={code}".format(code=code) 
    print("요청 URL = {}".format(url)) 
    return url 
# 조회하고 싶은 종목 일자데이터 url 가져오기 
item_name='카카오' 
url = get_url(item_name, code_df) 
# 일자 데이터를 담을 df라는 DataFrame 정의 
df = pd.DataFrame() 
# 1페이지에서 20페이지의 데이터만 가져오기 
for page in range(1, 21): 
    pg_url ='{url}&page={page}'.format(url=url, page=page) 
    df = df.append(pd.read_html(pg_url, header=0)[0], ignore_index=True) 
# df.dropna()를 이용해 결측값 있는 행 제거 
df = df.dropna() 
# 상위 5개 데이터 확인하기 
df.head()


요청 URL = http://finance.naver.com/item/sise_day.nhn?code=035720


,날짜,종가,전일비,시가,고가,저가,거래량
1,2020.05.20,226000.0,6000.0,223500.0,226500.0,222000.0,467088.0
2,2020.05.19,220000.0,1000.0,221500.0,222500.0,214000.0,1308903.0
3,2020.05.18,221000.0,500.0,221500.0,225500.0,218500.0,744533.0
4,2020.05.15,221500.0,3500.0,228500.0,231500.0,216000.0,1524083.0
5,2020.05.14,225000.0,8500.0,216000.0,228000.0,214000.0,1459813.0


In [4]:
# 한글로 된 컬럼명을 영어로 바꿔줌 
df = df.rename(columns= {'날짜': 'date', '종가': 'close', '전일비': 'diff', '시가': 'open', '고가': 'high', '저가': 'low', '거래량': 'volume'}) 
# 데이터의 타입을 int형으로 바꿔줌 
df[['close', 'diff', 'open', 'high', 'low', 'volume']] =df[['close', 'diff', 'open', 'high', 'low', 'volume']].astype(int) 
# 컬럼명 'date'의 타입을 date로 바꿔줌 
df['date'] = pd.to_datetime(df['date']) 
# 일자(date)를 기준으로 오름차순 정렬 
df = df.sort_values(by=['date'], ascending=True) 
# 상위 5개 데이터 확인 
df.head()


,date,close,diff,open,high,low,volume
298,2019-07-26,131000,0,129000,132000,127000,701403
297,2019-07-29,129000,2000,130000,132000,128000,368682
296,2019-07-30,126000,3000,128500,129500,126000,467439
295,2019-07-31,127000,1000,125000,128000,124500,294609
294,2019-08-01,126500,500,126000,128500,125500,249194


In [ ]:
# 설치 후엔 주석처리하여도 무방 
!pip install plotly

In [5]:
import plotly.offline as offline 
import plotly.graph_objs as go 
# jupyter notebook 에서 출력
offline.init_notebook_mode(connected=True) 
trace = go.Scatter( x=df.date, y=df.close, name=item_name) 
data = [trace] 
layout = dict( title='{}의 종가(close) Time Series'.format(item_name), 
              xaxis=dict( 
                  rangeselector=dict( 
                      buttons=list([ 
                          dict(count=1, label='1m', step='month', stepmode='backward'), 
                          dict(count=3, label='3m', step='month', stepmode='backward'),
                          dict(count=6, label='6m', step='month', stepmode='backward'), 
                          dict(step='all') 
                      ]) 
                  ), 
                  rangeslider=dict(), 
                  type='date' 
              ) 
             ) 
fig = go.Figure(data=data, layout=layout) 
offline.iplot(fig)